### 문제 정의: 순차적인 Python 코드
Dask의 효용성을 확인하기 위해, 의도적으로 시간을 소모하는 간단한 함수를 정의

In [ ]:
import time


def task_A():
    print("A: 작업 시작")
    time.sleep(1)  # 1초간 I/O 대기 (e.g., 파일 읽기)
    print("A: 작업 완료")
    return 1


def task_B():
    print("B: 작업 시작")
    time.sleep(1)  # 1초간 I/O 대기
    print("B: 작업 완료")
    return 2


def combine(a, b):
    print("C: 취합 시작")
    time.sleep(1)  # 1초간 연산 (e.g., 데이터 처리)
    print("C: 취합 완료")
    return a + b

In [ ]:
%%time

a = task_A()
b = task_B()
c = combine(a, b)

# task_A와 task_B는 서로 의존성이 없음에도 불구하고, Python의 특성상 순차적으로 실행됩니다.
print(f"최종 결과: {c}")

### Dask의 해법: dask.delayed
dask.delayed는 함수를 '지연'시키는 데코레이터 또는 래퍼(wrapper)입니다. delayed로 감싸진 함수는 즉시 실행되지 않고, Dask가 관리하는 'Delayed 객체'를 반환합니다.

In [ ]:
%%time
from dask import delayed

lazy_a = delayed(task_A)()
lazy_b = delayed(task_B)()
lazy_c = delayed(combine)(lazy_a, lazy_b)

# 이 객체는 어떤 함수를 어떤 인자로 실행해야 하는지에 대한 정보와
print(f"지연 객체 a: {lazy_a}")
# 다른 Delayed 객체와의 의존성을 담고 있습니다. (lazy_c는 lazy_a와 lazy_b가 필요함)
print(f"지연 객체 c: {lazy_c}")

### 태스크 그래프 시각화

In [ ]:
lazy_c.visualize()

### 계획의 실행: .compute()와 스케줄러

In [ ]:
%%time

print("--- Dask 연산 시작 ---")
result = lazy_c.compute()
print("--- Dask 연산 완료 ---")

# 총 실행 시간이 2초로 단축되었습니다.
# Dask 스케줄러가 태스크 그래프를 분석하여 의존성이 없는 task_A와 task_B를 서로 다른 스레드에 할당하여 동시에 실행했기 때문입니다.
print(f"최종 결과: {result}")

### 스케줄러 명시적 지정

In [ ]:
lazy_c.compute(scheduler="threads")  # 기본값
lazy_c.compute(scheduler="processes")  # 프로세스 기반 병렬 처리.
lazy_c.compute(scheduler="sync")  # 3초가 걸립니다. (병렬 X)